In [0]:
dataset = sc.textFile('/FileStore/tables/friendRecommenderDataset.txt')
dataset.take(10)

Out[1]: ['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878',
 '3\t0,12,41,55,1532,12636,13185,27552,38737',
 '4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822',
 '5\t0,1,20,2022,22939,23527,30257,32503,35633,41457,43262,44846,49574,31140,32828',
 '6\t0,21,98

In [0]:
# The dataset list originally has elements in the format: userId <tab> list of friends, separated by commas
# To convert the list of strings into key value pairs, the string first needs to be split by the tab character, 
# to separate the user and the list of friends. Then the string with the list of friends can be split by the comma
# The friends are still strings, so for each friendID in the list, if the string is numeric the string is converted to an integer.
userAndFriends = dataset.map(lambda x: x.split('\t')).map(lambda x: (int(x[0]), x[1].split(','))).mapValues(lambda x: [int(friendID) for friendID in x if friendID.isnumeric()])
print(userAndFriends.take(10))

[(0, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]), (1, [0, 5, 20, 135, 2409, 8715, 8932, 10623, 12347, 12846, 13840, 13845, 14005, 20075, 21556, 22939, 23520, 28193, 29724, 29791, 29826, 30691, 31232, 31435, 32317, 32489, 34394, 35589, 35605, 35606, 35613, 35633, 35648, 35678, 38737, 43447, 44846, 44887, 49226, 49985, 623, 629, 4999, 6156, 13912, 14248, 15190, 17636, 19217, 20074, 27536, 29481, 29726, 29767, 30257, 33060, 34250, 34280, 34392, 34406, 34418, 34420, 34439, 34450, 34651, 45054, 49592]), (2, [0, 117, 135, 1220, 2755, 12453, 24539, 24714, 41456, 45046, 49927, 6893, 13795, 16659, 32828, 41878]), (3, [0, 12, 41, 55, 1532, 12636, 13185, 27552, 38737]), (4, [0,

In [0]:
userFriendsMap = userAndFriends.collectAsMap()

userAndNotFriends = userAndFriends.flatMap(lambda x: [(x[0], [userId for userId in userFriendsMap[friend] if userId not in x[1] and userId != x[0]]) for friend in x[1]]).reduceByKey(lambda x, y: x + y).mapValues(lambda x: [userid for userid in set(x)])
#print(userAndNotFriends.take(2))
userAndNotFriends.cache()

randomUserSample = userAndFriends.takeSample(withReplacement=False, num=10)
randomUserIds = [randUserId for randUserId, _ in randomUserSample]

for randUser in randomUserIds:
    curUserNotFriends = userAndNotFriends.filter(lambda x: x[0] == randUser)
    rankedRecFriends = curUserNotFriends.flatMap(lambda x: [(userId, len((set(userFriendsMap[randUser]) & set(userFriendsMap[userId])))) \
                            for userId in x[1] if userId != randUser and userId not in userFriendsMap[randUser]])
    distinctRecs = rankedRecFriends.distinct().sortBy(lambda x: -x[1])
    top10Recs = distinctRecs.keys().take(10)
    print(randUser, end='\t')
    if top10Recs:
        for idx in range(len(top10Recs) - 1):
            print(top10Recs[idx], end=',')
        print(top10Recs[-1])
    else:
        print()


903	894,890,898,884,892,901,897,899,905,907
6852	6851,16652,20932,3412,28214,24258,8042,13016,24723,6855
6840	6839,4206,4246,13430,4234,10987,13263,34825,621,2865
34624	22756,19428,34266,34439,34364,34326,19427,34427,11395,34179
22102	4048,22118,22158,10987,22091,22161,16211,4264,11536,48920
1619	1537,1539,1541,1543,1545,1547,1549,1551,1553,1555
40633	49621,49606,49652,49698,41069,49691,49693,9453,13119,5531
22307	9170,26872,2680,43123,43185,26877,43349,5005,43190,43242
40769	40771,40772,40754,40762,37889,40755,40763,40775,39169,41733
27972	1415,27919,27921,27923,27925,27927,27929,27931,27933,27935


In [0]:
!pip install --upgrade pip
!pip install nltk

     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.3/773.3 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.0 MB/s eta 0:00:00


In [0]:
# Text classification dataset: eCommerce product classification
# Link to the dataset: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification/
textData = spark.read.csv('/FileStore/tables/ecommerceDataset.csv')
print(textData.count())
curDataset = textData.dropna()
print(curDataset.count())
eCommerceRDD = curDataset.rdd.map(tuple)
eCommerceRDD.take(10)

50456
50448
Out[2]: [('Household',
  'Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are f

In [0]:
filteredItems = eCommerceRDD.filter(lambda x: x[0] in ['Household', 'Books', 'Clothing & Accessories', 'Electronics'])
itemClasses = filteredItems.keys().collect()
set(itemClasses)

Out[3]: {'Books', 'Clothing & Accessories', 'Electronics', 'Household'}

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Out[4]: True

In [0]:
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# clean the item descriptions
# Remove stop words, punctuation, destem
# https://www.nltk.org/api/nltk.stem.wordnet.html
# https://towardsdatascience.com/laplace-smoothing-in-na%C3%AFve-bayes-algorithm-9c237a8bdece
# https://towardsdatascience.com/understanding-na%C3%AFve-bayes-algorithm-f9816f6f74c0
# https://www.guru99.com/stemming-lemmatization-python-nltk.html 
# https://towardsdatascience.com/introduction-to-na%C3%AFve-bayes-classifier-fa59e3e24aaf
# https://towardsdatascience.com/unfolding-na%C3%AFve-bayes-from-scratch-2e86dcae4b01
# https://medium.com/udemy-engineering/pyspark-under-the-hood-randomsplit-and-sample-inconsistencies-examined-7c6ec62644bc

#lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
stopWords = list(stopwords.words('english'))

def cleanText(text):

    text = ''.join([ch for ch in text if ch not in string.digits and ch not in string.punctuation])
    words = nltk.word_tokenize(text.lower())
    words = [word for word in words if word not in stopWords]
    # words = [lemmatizer.lemmatize(word, pos='n') for word in words]
    words = [ps.stem(word) for word in words]
    return words

cleanedDescriptions = filteredItems.mapValues(lambda x: cleanText(x))
print(cleanedDescriptions.take(5))
trainData, testData = cleanedDescriptions.randomSplit([0.75, 0.25], seed=None)
trainData = trainData.cache()
testData = testData.cache()
trainData.take(10)

[('Household', ['paper', 'plane', 'design', 'frame', 'wall', 'hang', 'motiv', 'offic', 'decor', 'art', 'print', 'x', 'inch', 'set', 'paint', 'made', 'synthet', 'frame', 'uv', 'textur', 'print', 'give', 'multi', 'effect', 'attract', 'toward', 'special', 'seri', 'paint', 'make', 'wall', 'beauti', 'give', 'royal', 'touch', 'paint', 'readi', 'hang', 'would', 'proud', 'possess', 'uniqu', 'paint', 'nich', 'apart', 'use', 'modern', 'effici', 'print', 'technolog', 'print', 'ink', 'precis', 'epson', 'roland', 'hp', 'printer', 'innov', 'hd', 'print', 'techniqu', 'result', 'durabl', 'spectacular', 'look', 'print', 'highest', 'last', 'lifetim', 'print', 'sole', 'topnotch', 'ink', 'achiev', 'brilliant', 'true', 'colour', 'due', 'high', 'level', 'uv', 'resist', 'print', 'retain', 'beauti', 'colour', 'mani', 'year', 'add', 'colour', 'style', 'live', 'space', 'digit', 'print', 'paint', 'pleasur', 'etern', 'blissso', 'bring', 'home', 'eleg', 'print', 'lush', 'rich', 'color', 'make', 'noth', 'sheer', 'e

In [0]:
itemClassAndNumRows = trainData.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
itemClassCountMap = itemClassAndNumRows.collectAsMap()
itemClassCountMap

Out[6]: {'Electronics': 7987,
 'Household': 14426,
 'Books': 8894,
 'Clothing & Accessories': 6544}

In [0]:
numTrainData = trainData.count()
numTestData = testData.count()
itemClassProbabilitiesTrain = itemClassAndNumRows.mapValues(lambda x: x / numTrainData).collectAsMap()
itemClassProbabilitiesTrain

Out[7]: {'Electronics': 0.2110115981083723,
 'Household': 0.3811259940292198,
 'Books': 0.23497397690946079,
 'Clothing & Accessories': 0.1728884309529471}

In [0]:
import numpy as np

trainDataUniqueWordsPerClass = trainData.reduceByKey(lambda x, y: x + y).mapValues(lambda x: list(set(x)))
wordCountTrain = trainData.flatMap(lambda x: [((x[0], word), 1) for word in x[1]]).reduceByKey(lambda x, y: x + y)
wordCountTrain.take(20)

Out[8]: [(('Household', 'paper'), 737),
 (('Household', 'wall'), 2717),
 (('Household', 'offic'), 1520),
 (('Household', 'inch'), 2601),
 (('Household', 'possess'), 47),
 (('Household', 'brilliant'), 27),
 (('Household', 'retain'), 157),
 (('Household', 'add'), 1062),
 (('Household', 'cm'), 3565),
 (('Household', 'qualiti'), 3907),
 (('Household', 'sanfsw'), 2),
 (('Household', 'india'), 1118),
 (('Household', 'wooden'), 1280),
 (('Household', 'birthday'), 189),
 (('Household', 'natur'), 1076),
 (('Household', 'laser'), 175),
 (('Household', 'need'), 1926),
 (('Household', 'text'), 24),
 (('Household', 'email'), 57),
 (('Household', 'life'), 1090)]

In [0]:
wordToClassProbMap = wordCountTrain.collectAsMap()
wordToClassProbMap

Out[9]: {('Household', 'paper'): 737,
 ('Household', 'wall'): 2717,
 ('Household', 'offic'): 1520,
 ('Household', 'inch'): 2601,
 ('Household', 'possess'): 47,
 ('Household', 'brilliant'): 27,
 ('Household', 'retain'): 157,
 ('Household', 'add'): 1062,
 ('Household', 'cm'): 3565,
 ('Household', 'qualiti'): 3907,
 ('Household', 'sanfsw'): 2,
 ('Household', 'india'): 1118,
 ('Household', 'wooden'): 1280,
 ('Household', 'birthday'): 189,
 ('Household', 'natur'): 1076,
 ('Household', 'laser'): 175,
 ('Household', 'need'): 1926,
 ('Household', 'text'): 24,
 ('Household', 'email'): 57,
 ('Household', 'life'): 1090,
 ('Household', 'stay'): 321,
 ('Household', 'memori'): 187,
 ('Household', 'pitaara'): 6,
 ('Household', 'romant'): 67,
 ('Household', 'canva'): 160,
 ('Household', 'breathtak'): 8,
 ('Household', 'galleri'): 19,
 ('Household', 'hotel'): 364,
 ('Household', 'surround'): 159,
 ('Household', 'fabric'): 1268,
 ('Household', 'giant'): 8,
 ('Household', 'onlin'): 375,
 ('Household', 's

In [0]:
import math
trainDataWordVocabulary = wordCountTrain.map(lambda x: x[0][1]).distinct().collect()
wordProbTrain = wordCountTrain.map(lambda x: ((x[0][0], x[0][1]), math.log((x[1] + 1) / (itemClassCountMap[x[0][0]] + len(trainDataWordVocabulary)))))
wordProbTrain.take(20)

Out[10]: [(('Household', 'paper'), -4.694351744820751),
 (('Household', 'wall'), -3.3906439747101342),
 (('Household', 'offic'), -3.9711722771619304),
 (('Household', 'inch'), -3.4342599103487728),
 (('Household', 'possess'), -7.427094558513333),
 (('Household', 'brilliant'), -7.966091059246019),
 (('Household', 'retain'), -6.235700536394257),
 (('Household', 'add'), -4.329445191079276),
 (('Household', 'cm'), -3.1190957709981375),
 (('Household', 'qualiti'), -3.02751455625855),
 (('Household', 'sanfsw'), -10.199683280753113),
 (('Household', 'india'), -4.278104861109298),
 (('Household', 'wooden'), -4.14289926752449),
 (('Household', 'birthday'), -6.051271497260737),
 (('Household', 'natur'), -4.316360892264835),
 (('Household', 'laser'), -6.127811574383071),
 (('Household', 'need'), -3.734575901006857),
 (('Household', 'text'), -8.079419744553023),
 (('Household', 'email'), -7.237852558874804),
 (('Household', 'life'), -4.303445583588153)]

In [0]:
# Use the test set on the trained word probabilities
wordProbTrainMap = wordProbTrain.collectAsMap()

def getClassProbs(itemDescription, itemClassProbabilitiesTrain, itemClassCountMap, wordProbTrainMap, trainDataWordVocabulary):
    classAndProb = []
    for itemClass, prob in itemClassProbabilitiesTrain.items():
        descriptionProb = np.log(prob)
        for word in itemDescription:
            testWordProb = 0
            if (itemClass, word) in wordProbTrainMap:
                testWordProb += wordProbTrainMap.get((itemClass, word))
            else:
                testWordProb += math.log(1 / (itemClassCountMap[itemClass] + len(trainDataWordVocabulary)))
            descriptionProb += testWordProb
        classAndProb.append((itemClass, descriptionProb))
    return classAndProb

testDataClassProbs = testData.mapValues(lambda x: (x, getClassProbs(x, itemClassProbabilitiesTrain, itemClassCountMap, wordProbTrainMap, trainDataWordVocabulary)))
print(testDataClassProbs.take(5))
testPreds = testDataClassProbs.map(lambda x: (x[0], x[1][1]))
print(testPreds.take(5))
testAccuracy = testPreds.filter(lambda x: x[0] == max(x[1], key=lambda x: x[1])[0]).count() / numTestData
print("Naive Bayes model accuracy: ", testAccuracy)

[('Household', (['paper', 'plane', 'design', 'starri', 'night', 'vangoh', 'wall', 'art', 'canva', 'paint', 'larg', 'size', 'roll', 'canva', 'art', 'print', 'x', 'use', 'modern', 'effici', 'print', 'technolog', 'canvas'], [('Electronics', -145.57126322444617), ('Household', -120.45210491152699), ('Books', -155.1501906973765), ('Clothing & Accessories', -165.68896539283466)])), ('Household', (['paint', 'villa', 'uv', 'textur', 'modern', 'art', 'print', 'frame', 'paint', 'home', 'decor', 'size', 'x', 'x', 'cm', 'paint', 'villa', 'uv', 'textur', 'modern', 'art', 'print', 'frame', 'paint', 'home', 'decor', 'size', 'x', 'x', 'cm', 'decor', 'home', 'offic', 'wall', 'artist', 'design', 'satin', 'matt', 'textur', 'frame', 'uv', 'art', 'print', 'mav', 'design', 'reflect', 'artist', 'qualiti', 'time', 'one', 'taken', 'design', 'poster', 'exquisit', 'print', 'frame', 'also', 'perfect', 'gift', 'option', 'colleagu', 'love', 'one', 'famili', 'member', 'frame', 'dont', 'includ', 'glass', 'light', 'we

In [0]:
# Filter test data for each item class
householdTest = testPreds.filter(lambda x: x[0] == 'Household')
clothingTest = testPreds.filter(lambda x: x[0] == 'Clothing & Accessories')
electronicsTest = testPreds.filter(lambda x: x[0] == 'Electronics')
booksTest = testPreds.filter(lambda x: x[0] == 'Books')

householdCount = householdTest.count()
clothingCount = clothingTest.count()
electronicsCount = electronicsTest.count()
booksCount = booksTest.count()

print("Household count: ", householdCount)
print("Clothing & Accessories count: ", clothingCount)
print("Electronics count: ", electronicsCount)
print("Books count: ", booksCount)

print()

householdAccuracy = householdTest.filter(lambda x: x[0] == max(x[1], key=lambda x: x[1])[0]).count() / householdCount
print("Household accuracy: ", householdAccuracy)

clothingAccuracy = clothingTest.filter(lambda x: x[0] == max(x[1], key=lambda x: x[1])[0]).count() / clothingCount
print("Clothing & Accessories accuracy: ", clothingAccuracy)

electronicsAccuracy = electronicsTest.filter(lambda x: x[0] == max(x[1], key=lambda x: x[1])[0]).count() / electronicsCount
print("Electronics accuracy: ", electronicsAccuracy)

booksAccuracy = booksTest.filter(lambda x: x[0] == max(x[1], key=lambda x: x[1])[0]).count() / booksCount
print("Books accuracy: ", booksAccuracy)

Household count:  4887
Clothing & Accessories count:  2126
Electronics count:  2634
Books count:  2926

Household accuracy:  0.9817884182525066
Clothing & Accessories accuracy:  0.8245531514581373
Electronics accuracy:  0.9009111617312073
Books accuracy:  0.9121667805878332
